In [66]:
def get_input() -> list[str]:
    return get_lines_from_file('./input')

def get_test_input() -> list[str]:
    return get_lines_from_file('./test_input')

def get_lines_from_file(filepath) -> list[str]:
    with open(filepath) as f:
        return [line.strip('\n') for line in f.readlines()]

def get_str_from_file(filepath) -> str:
    with open(filepath) as f:
        return f.readline().strip('\n')

def get_int_from_file(filepath) -> int:
    with open(filepath) as f:
        return int(f.readline().strip())

def log_invocation(func):
    def logged_func(*args):
        res = func(*args)
        print(f'{func.__name__}({args}) -> {res}')
        return res
    return logged_func

In [67]:
from typing import NamedTuple
from itertools import chain
import networkx as nx

class Cell(NamedTuple):
    x: int
    y: int
    value: str

def is_adjacent(cell1: Cell, cell2: Cell) -> bool:
    if cell1.x == cell2.x:
        return cell1.y == cell2.y+1 or cell1.y == cell2.y-1
    if cell1.y == cell2.y:
        return cell1.x == cell2.x+1 or cell1.x == cell2.x-1

def is_reachable(start: Cell, dest: Cell) -> bool:
    return any((
        dest.value < start.value,
        dest.value == start.value,
        dest.value == chr(ord(start.value)+1)
    ))

def get_neighbors(grid: Cell, cell: Cell) -> list[Cell]:
    nbs = []
    width = len(grid[0])
    height = len(grid)
    x, y = cell.x, cell.y
    if x > 0:
        nbs.append(grid[y][x-1])
    if x < width-1:
        nbs.append(grid[y][x+1])
    if y > 0:
        nbs.append(grid[y-1][x])
    if y < height-1:
        nbs.append(grid[y+1][x])
    return nbs

def parse_input(input: list[str]) -> list[Cell]:
    return [
        [Cell(x, y, value) for x, value in enumerate(line)] 
        for y, line in enumerate(input)
    ]

def find_cell_value(cells: list[Cell], value: str) -> Cell:
    return next(filter(lambda c: c.value == value, cells))

def flatten(l) -> list[Cell]:
    return list(chain(*l))

def solution1(input: list[str]) -> int:
    field_width = len(input[0])
    field_height = len(input)
    grid = parse_input(input)
    temp_start = find_cell_value(flatten(grid), 'S')
    temp_dest = find_cell_value(flatten(grid), 'E')

    start_cell = Cell(temp_start.x, temp_start.y, 'a')
    dest_cell = Cell(temp_dest.x, temp_dest.y, 'z')

    grid[temp_start.y][temp_start.x] = start_cell
    grid[temp_dest.y][temp_dest.x] = dest_cell

    g = nx.DiGraph()
    for cell in flatten(grid):
        g.add_edges_from([(cell, dest) for dest in get_neighbors(grid, cell) if is_reachable(cell, dest)])
    
    sp = nx.shortest_path(g, start_cell, dest_cell)
    
    return len(sp)-1


In [68]:

def solution2(input: list[str]) -> int:
    field_width = len(input[0])
    field_height = len(input)
    grid = parse_input(input)
    temp_start = find_cell_value(flatten(grid), 'S')
    temp_dest = find_cell_value(flatten(grid), 'E')

    start_cell = Cell(temp_start.x, temp_start.y, 'a')
    dest_cell = Cell(temp_dest.x, temp_dest.y, 'z')

    grid[temp_start.y][temp_start.x] = start_cell
    grid[temp_dest.y][temp_dest.x] = dest_cell

    start_cells = list(filter(lambda c: c.value == 'a', flatten(grid)))

    g = nx.DiGraph()
    for cell in flatten(grid):
        g.add_edges_from([(cell, dest) for dest in get_neighbors(grid, cell) if is_reachable(cell, dest)])
    
    sp = []
    for start in start_cells:
        try:
            sp.append(len(nx.shortest_path(g, start, dest_cell))-1)
        except:
            pass
    
    return min(sp)

In [69]:
solutions = [
    solution1,
    solution2,
]

test_results = [
    get_int_from_file('./test_result1'),
    get_int_from_file('./test_result2'),
]

def run_test(idx) -> bool:
    res = solutions[idx-1](get_test_input())
    test_res = test_results[idx-1]
    
    if test_res == res:
        print(f'Your solution for part {idx} works!!! :) (on the test input, that is)')
        print(f'Let`s try it on the actual input now...')
        return True
    else:
        print(f'Your solution for part {idx} does not work yet. Keep going!')
        print(f'You`ve got {res}, but the correct test result is {test_res}')
        return False

def run_solution(idx):
    sol = solutions[idx-1](get_input())
    print(f'The solution for part {idx} is: {sol}')

if run_test(1):
    run_solution(1)
    print('\nOn to part 2...\n')
    if run_test(2):
        run_solution(2)

Your solution for part 1 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 1 is: 370

On to part 2...

Your solution for part 2 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 2 is: 363
